In [1]:
import pandas as pd
import glob
import os
from tqdm import tqdm as tqdm
import time

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
from datetime import timedelta
import glob
import os
from tqdm import tqdm as tqdm
import time

import geopandas
import geopy
from geopy.geocoders import Nominatim
import folium
from geopy.extra.rate_limiter import RateLimiter
from folium import plugins
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

% matplotlib inline
import statsmodels.api as sm
from statsmodels.compat import lzip
from statsmodels.formula.api import ols
from sklearn.ensemble import GradientBoostingRegressor


def getAllFiles(path, file_name_type, start_year, start_month, end_year, end_month, dropped_columns):
    li = []

    for x in tqdm(list(os.walk(data_path))):

        all_files = glob.glob(x[0] + "/*" + file_name_type + ".csv")

        for file_path in all_files:
            if (inTimePeriod(file_path, start_year, start_month, end_year, end_month)):
                df = pd.read_csv(file_path, index_col=None, header=0)
                df.drop(columns=dropped_columns, inplace=True)
                df['Year'] = df['Month'].apply(monthYearToYear)
                df['Month'] = df['Month'].apply(monthYearToMonth)
                df = df[df['Longitude'].notna()]
                df = df[df['Latitude'].notna()]
                li.append(df)

    return pd.concat(li, axis=0, ignore_index=True)


def inTimePeriod(file_path, start_year, start_month, end_year, end_month):
    file_path_parts = file_path.split("\\")
    file_name = file_path_parts[len(file_path_parts) - 1]
    year = int(file_name[:4])
    month = int(file_name[5:7])
    start_bool = (year >= start_year) or (year == start_year and month >= start_month)
    end_bool = (year <= end_year) or (year == end_year and month <= end_month)
    if (start_bool and end_bool):
        return True
    return False


def monthYearToYear(monthYear):
    return int(monthYear[:4])


def monthYearToMonth(monthYear):
    return int(monthYear[5:7])


def MonthYearRegionDF(dataframe, month, year, region):
    return dataframe[(dataframe['Month'] == month) & (dataframe['Year'] == year) &
                     (dataframe['Falls within'] == region)]


data_path = r"C:\Users\20201935\Documents\Data Challenge 2\Jan_2010_Oct_2021"

fullStreetFrame = getAllFiles(data_path, "street", 2012, 1, 2021, 10,
                              ['Context', 'Location', 'Reported by', 'Last outcome category', 'LSOA code',
                               'LSOA name'])

test = MonthYearRegionDF(fullStreetFrame, 1, 2017, 'Avon and Somerset Constabulary')
test2 = test[test['Crime type'] == 'Violence and sexual offences']
test2.head(1)
## Heatmaps
hm_markers = folium.Map([51.4549741, -2.5979205], zoom_start=14)
for index, row in tqdm(test2.iterrows()):
    folium.CircleMarker([row['Latitude'], row['Longitude']],
                        radius=3,
                        popup=row['Crime type'],
                        fill_color="#3db7e4",  # divvy color
                        ).add_to(hm_markers)

heat_data = [[row['Latitude'], row['Longitude']] for index, row in test2.iterrows()]
HeatMap(heat_data).add_to(hm_markers)
hm_markers
heat_data = [[row['Latitude'], row['Longitude']] for index, row in test2.iterrows()]

hm = folium.Map([51.4549741, -2.5979205], zoom_start=14)

# Plot it on the map
HeatMap(heat_data).add_to(hm)

hm



In [ ]:
data_path = r"C:\Users\20202810\OneDrive - TU Eindhoven\Documents\University\Year 2\JGB050 - Data Challenge 2\Jan_2010_Oct_2021"

#trainstreetframe = getAllFiles(data_path, "street", 2012, 1, 2018, 10)
#teststreetframe = getAllFiles(data_path, "street", 2018, 11, 2021, 10)
fullStreetFrame = getAllFiles(data_path, "street", 2012, 1, 2021, 10, ['Context', 'Location', 'Reported by', 'Last outcome category', 'LSOA code', 'LSOA name'])

100%|██████████| 132/132 [11:22<00:00,  5.17s/it]


In [ ]:
fullStreetFrame

In [ ]:
fullStreetFrame['LSOA name'].isna().sum()

In [ ]:
trainstreetframe.to_csv("train_street_data.csv")
trainstreetframe

In [ ]:
teststreetframe.to_csv("test_street_data.csv")
teststreetframe

In [ ]:
trainstreetframe.groupby(['Crime type']).size()

In [ ]:
trainstreetframe.groupby(['Reported by', 'Falls within']).size()

In [ ]:
trainstreetframe = trainstreetframe.reset_index() 

for index, row in trainstreetframe.iterrows():
    print(index)
    if (row['Reported by'] != row['Falls within']):
        print(row['Reported by'] + " " + row['Falls within'])
        print(trainstreetframe.loc)
        break

In [ ]:
#testframe = getAllFiles(data_path, "street", 2020, 11, 2021, 10, ['Context', 'Location', 'Reported By', 'Last outcome category', 'LSOA code', 'LSOA name'])

In [ ]:
testframe.groupby(['Reported by', 'Falls within']).size()

In [ ]:
falls_within = testframe['Falls within'].unique()

In [ ]:
falls_within[0]